In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/15859/matrix-factorization

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/15859/matrix-factorization


In [2]:
import numpy as np
import pandas as pd

In [4]:
filename = 'data/ml-latest-small/ratings.csv'
df = pd.read_csv(filename)
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [6]:
A = np.zeros((193609, 610))
for index, row in df.iterrows():
    A[int(row['movieId']-1), int(row['userId']-1)] = row['rating']
with open('data/A.npy', 'wb') as f:
    np.save(f, A)

In [3]:
with open('data/A.npy', 'rb') as f:
    A = np.load(f)

In [6]:
def low_rank_best(A, k):
    U, S, VT = np.linalg.svd(A, full_matrices=False)
    X = U[:, :k] @ np.diag(S[:k]) @ VT[:k, :]
    error = np.linalg.norm(A - X) ** 2
    print(error)
    return X, error

array([[ 2.86172577e+00,  1.92521651e-01,  3.16221539e-02, ...,
         2.80950595e+00,  8.33535845e-01,  8.14654722e-01],
       [ 9.37778016e-01, -8.26445812e-03,  1.65707762e-02, ...,
         1.98035888e+00,  6.58873461e-01,  2.74452616e+00],
       [ 9.75957138e-01, -2.69977785e-02,  1.95043206e-02, ...,
         1.80789812e+00,  3.08578230e-01, -2.78495459e-01],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.93090709e-02,  1.31879571e-02, -1.88893510e-03, ...,
        -3.26583065e-03,  8.35869708e-04,  6.41280361e-02]])

In [7]:
X, error = low_rank_best(A, 10)

834691.5219530301


In [9]:
def _als_step(ratings, solve_vecs, fixed_vecs):
    A = fixed_vecs.T.dot(fixed_vecs)
    b = ratings.dot(fixed_vecs)
    A_inv = np.linalg.inv(A)
    solve_vecs = b.dot(A_inv)
    return solve_vecs

def alternating_least_square(A, k, num_iters):
    n, d = A.shape
    U = np.random.random((n, k))
    V = np.random.random((d, k))
    error_list = []
    for i in range(num_iters):
        U = _als_step(A, U, V)
        V = _als_step(A.T, V, U)
        error = np.linalg.norm(A - U @ V.T) ** 2
        error_list.append(error)
        print(error)
    return U, V, error_list

In [10]:
U, V, error_list = alternating_least_square(A, 10, 100)

1000531.1961694105
877574.0237994391
852495.2474602364
844182.3189492968
840754.4270307682
839240.2591021283
838461.1280787289
837976.012169797
837608.3183786665
837278.4518989755
836949.3577350876
836609.2476236499
836264.80727454
835934.2387163965
835637.8727049177
835389.7705354623
835194.2385113917
835047.4020297029
834941.0032112217
834865.7756629966
834813.402699386
834777.2442117706
834752.3521157731
834735.192202018
834723.3054633997
834715.0080001288
834709.1571494362
834704.9809606419
834701.9585001814
834699.7377184596
834698.0798544716
834696.8221045337
834695.8526642189
834695.0940736734
834694.4921051157
834694.0083379947
834693.6151860935
834693.2925537504
834693.0255777723
834692.8030924693
834692.6165793173


KeyboardInterrupt: ignored

In [ ]:
class ALS:
    """
    Train a matrix factorization model using Alternating Least Squares
    to predict empty entries in a matrix
    
    Parameters
    ----------
    n_iters : int
        number of iterations to train the algorithm
        
    n_factors : int
        number of latent factors to use in matrix 
        factorization model, some machine-learning libraries
        denote this as rank
        
    reg : float
        regularization term for item/user latent factors,
        since lambda is a keyword in python we use reg instead
    """

    def __init__(self, n_iters, n_factors, reg):
        self.reg = reg
        self.n_iters = n_iters
        self.n_factors = n_factors  
        
    def fit(self, train, test):
        """
        pass in training and testing at the same time to record
        model convergence, assuming both dataset is in the form
        of User x Item matrix with cells as ratings
        """
        self.n_user, self.n_item = train.shape
        self.user_factors = np.random.random((self.n_user, self.n_factors))
        self.item_factors = np.random.random((self.n_item, self.n_factors))
        
        # record the training and testing mse for every iteration
        # to show convergence later (usually, not worth it for production)
        self.test_mse_record  = []
        self.train_mse_record = []   
        for _ in range(self.n_iters):
            self.user_factors = self._als_step(train, self.user_factors, self.item_factors)
            self.item_factors = self._als_step(train.T, self.item_factors, self.user_factors) 
            predictions = self.predict()
            test_mse = self.compute_mse(test, predictions)
            train_mse = self.compute_mse(train, predictions)
            self.test_mse_record.append(test_mse)
            self.train_mse_record.append(train_mse)
        
        return self    
    
    def _als_step(self, ratings, solve_vecs, fixed_vecs):
        """
        when updating the user matrix,
        the item matrix is the fixed vector and vice versa
        """
        A = fixed_vecs.T.dot(fixed_vecs) + np.eye(self.n_factors) * self.reg
        b = ratings.dot(fixed_vecs)
        A_inv = np.linalg.inv(A)
        solve_vecs = b.dot(A_inv)
        return solve_vecs
    
    def predict(self):
        """predict ratings for every user and item"""
        pred = self.user_factors.dot(self.item_factors.T)
        return pred
    
    @staticmethod
    def compute_mse(y_true, y_pred):
        """ignore zero terms prior to comparing the mse"""
        mask = np.nonzero(y_true)
        mse = mean_squared_error(y_true[mask], y_pred[mask])
        return mse

In [ ]:
class GD:
    